## Periodic boundary conditions

Periodic Green's function for simulation of infinite lattice. Use of symmetry factors for speed-up. Use a different file for each number of periodic boxes and Fourier modes used, in the format of 

    periodic_Nb_Nm.py

In [1]:
from symengine import *
import sympy
import numpy as np
import re
from joblib import Parallel, delayed
from joblib import parallel_backend

Number of periodic boxes and Fourier modes

In [2]:
Nb = 1
Nm = 4

Nbb = 2*Nb +1
N1 = int(-(Nm/2)+1)
N2 = int((Nm/2)+1)

In [3]:
Nbb, N1, N2

(3, -1, 3)

### Definitions and auxiliary functions

In [4]:
xij, yij, zij = symbols("xij, yij, zij", real=True) 
b, eta = symbols("b, eta", real=True, positive=True)

# indices for summation:
## Caution, cannot use lambda, instead use Lambda
## do not use eta as index, it's the viscosity
alpha, beta, gamma, gamma1, gamma2, gamma3, gamma4, kappa, kappa1, kappa2, mu, nu, nu1, nu2, Lambda, eta2, zeta, sigma = symbols(
    'alpha, beta, gamma, gamma1, gamma2, gamma3, gamma4, kappa, kappa1, kappa2, mu, nu, nu1, nu2, Lambda, eta2, zeta, sigma', integer=True)

#new symbols for periodic calculation: boxSize, xi (convergence parameter)
L, xi = symbols("L, xi", real=True, positive=True)
kx, ky, kz = symbols("kx, ky, kz", real=True) 

rij = Matrix([xij, yij, zij])

k0 = 2*pi/L

def norm(r):
    result=0
    for i in range(3):
        result += r[i]**2
    return sqrt(result)

KroneckerDelta = eye(3)

def eps(alpha, beta, gamma):
    return LeviCivita(alpha,beta,gamma)

### Green's function

In [5]:
##pass xi*norm(r) and norm(k) for A, B and C

def A(x): ##are these correct? Don't match Pozrikidis
    return erfc(x) + 2/sqrt(pi)*(2*x**3 - 3*x)*exp(-x**2)

def B(x):
    return erfc(x) + 2/sqrt(pi)*(x - 2*x**3)*exp(-x**2)

def C(k):
    return (S(1) + k**2/xi**2/4 + k**4/xi**4/8)*exp(-k**2/xi**2/4)  ## was exp(-1/4*k**4/xi**4) as in Rajesh' PhD thesis

def M1(xij, yij, zij, alpha,beta): ##factors of 8*pi?
    rij = Matrix([xij, yij, zij])
    return A(xi*norm(rij))*KroneckerDelta[alpha,beta]/norm(rij) + B(xi*norm(rij))*rij[alpha]*rij[beta]/norm(rij)**3

def M2(xij, yij, zij, kx, ky, kz, alpha,beta): ## k*rij is dot product
    rij = Matrix([xij, yij, zij])
    k = Matrix([kx, ky, kz]) 
    kDOTrij = xij*kx + yij*ky + zij*kz
    return 1/(eta)*C(norm(k))*(KroneckerDelta[alpha,beta]/norm(k)**2 - k[alpha]*k[beta]/norm(k)**4)*cos(kDOTrij) ##take the real part (?) follow Pozrikidis (3.14)
                                                                                                                ## just use cos instead of exp(I*kDOTrij)

##define function for periodic Green's function, evaluate it once and save it as another function, as to not have to evaluate it every time 
def GreenReal(alpha, beta): ##eventually evaluate xij=L
    green = 0

    for ii in range(Nbb):
        dx = (ii - Nb)*xij
        for jj in range(Nbb):
            dy = (jj - Nb)*yij
            for kk in range(Nbb):
                dz = (kk - Nb)*zij
                
                if ii==jj==kk: ##self-interaction added later by hand
                    pass
                else:
                    green += M1(dx, dy, dz, alpha, beta)
                    
    return green


def GreenFourier(alpha,beta): ##eventually evaluate xij=0 and take real part
    green = 0 
    
    for ii in range(N1, N2): 
        kx = k0*ii
        for jj in range(N1,N2):
            ky = k0*jj
            for kk in range(N1, N2):
                kz = k0*kk
                if kx != 0 or ky != 0 or kz != 0:
                    green += M2(xij, yij, zij, kx, ky, kz, alpha, beta)

    return green

In [6]:
##Take a closer look at GreenFourier

In [7]:
# GreenFourier(0,1)

In [8]:
# sympy.simplify(GreenReal(0,1).subs({xij:L, yij:L, zij:L}))

In [9]:
# sympy.simplify(GreenFourier(0,1).subs({xij:0, yij:0, zij:0}))

In [10]:
## First compute Green's functions and save in matrix & then in function, to not have to compute it every time (we fix the number of modes)
## then change below matrix elements computations to include an argument G, meaning which Green's function is used
##compute all matrix elements for both and then simply sum them GP = sum M1 + 1/V sum M2 - simplify too slow

In [11]:
matGR = zeros(3,3)
for i in range(3):
    for j in range(3):
        matGR[i,j] = GreenReal(i,j)
        
matGF = zeros(3,3)
for i in range(3):
    for j in range(3):
        matGF[i,j] = GreenFourier(i,j)
        
def GR(alpha, beta):
    return matGR[alpha,beta]

def GF(alpha,beta):
    return matGF[alpha,beta]

### Calculus auxiliary functions

In [12]:
def Laplacian(G):
    return diff(G, rij[0], rij[0]) + diff(G, rij[1], rij[1]) + diff(G, rij[2], rij[2])

def Curl(G, alpha, beta): ## the function to take the curl of and the two remaining indices 
    result=0
    for i in range(3):
        for j in range(3):
            result += eps(alpha,i,j)*diff(G(j,beta),rij[i])
    return result

def tilde_Curl(G, alpha, beta): ##introduce for convenience, minus sign already taken into account in matrix elements
    result=0
    for i in range(3):
        for j in range(3):
            result += eps(beta,i,j)*diff(G(alpha, j),rij[i])
    return result

def CurlCurl(G, alpha, beta): ## the function to take the curl of and the two remaining indices 
    result=0
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    result += eps(alpha,i,j)*eps(beta,k,l)*diff(G(j,l), rij[i], rij[k])
    return result


def SumG(G, sigma, eta2, mu):
    result=0
    for i in range(3):
        result += eps(sigma, eta2, i)*Laplacian( G(mu, i) )
    return result


def SumG3s1Sym(G, mu, nu2, alpha, gamma1, gamma2):
    result=0
    for i in range(3):
        result += eps(mu,nu2,i)*g3s1Sym(G, alpha,gamma1,gamma2,i) ##add argument G
    return result

In [13]:
##auxiliary functions
def delG(G, alpha, beta, gamma):
    return diff( G(alpha,beta), rij[gamma])

def lapG(G, alpha,beta):
    return Laplacian( G(alpha,beta) )

def dellapG(G, alpha, beta, kappa1):
    return diff(lapG(G, alpha, beta), rij[kappa1])

def deldelG(G, zeta, beta, kappa1, nu):
    return diff(delG(G, zeta, beta, kappa1), rij[nu])

def deldellapG(G, alpha, beta, gamma1, kappa1):
    return diff(dellapG(G, alpha,beta, gamma1), rij[kappa1])

def deldeldelG(G, alpha, beta, gamma1, gamma2, gamma3):
    return diff(deldelG(G, alpha, beta, gamma1, gamma2), rij[gamma3])

def deldeldeldelG(G, alpha, beta, gamma1, gamma2, gamma3, gamma4):
    return diff(deldeldelG(G, alpha, beta, gamma1, gamma2, gamma3), rij[gamma4])

In [14]:
# GLL  ##simplify later, when filling matrix elements, after subsituting for xij -- much faster
def G1s1s(G, alpha, beta):
    return G(alpha, beta) + b**2/3*lapG(G, alpha, beta)

def G1s2a(G, alpha, beta):
    return -b/2*tilde_Curl(G, alpha,beta)

def G2a1s(G, alpha, beta):
    return b*Curl(G, alpha,beta)

def G2a2a(G, alpha, beta):
    return -b**2/2*CurlCurl(G, alpha,beta)

In [15]:
## GL2s and G2sL
def G1s2s(G, alpha, kappa1, beta):
    return -b/2*((delG(G, alpha, beta, kappa1) + delG(G, alpha, kappa1, beta)) + 4*b**2/15*(dellapG(G, alpha, beta, kappa1) + dellapG(G, alpha, kappa1, beta)))

def G2a2s(G, alpha, kappa1, beta):
    return -b**2/2*(diff(Curl(G, alpha, beta),rij[kappa1]) + diff(Curl(G, alpha, kappa1),rij[beta]))

def G2s1s(G, alpha, gamma1, beta):
    return b/2*((delG(G, alpha, beta, gamma1) + delG(G, gamma1, beta, alpha)) + 4*b**2/15*(dellapG(G, alpha,beta,gamma1) + dellapG(G, gamma1, beta, alpha)))

def G2s2a(G, alpha, gamma1, mu):
    return -b**2/4*(diff(tilde_Curl(G, gamma1,mu), rij[alpha]) + diff(tilde_Curl(G, alpha,mu), rij[gamma1])) #sign change from -> eps(mu, kappa1, beta)


## G2s2s
def G2s2s(G, alpha, gamma1, kappa1, beta):
    return -b**2/4*((deldelG(G, alpha, beta, gamma1, kappa1) + deldelG(G, gamma1, beta, alpha, kappa1)) + 
                      (deldelG(G, alpha,kappa1, gamma1, beta) + deldelG(G, gamma1, kappa1, alpha, beta)) +
                      b**2/5*(deldellapG(G, alpha,beta,gamma1,kappa1) + deldellapG(G, gamma1,beta,alpha,kappa1)) + 
                      b**2/5*(deldellapG(G, alpha, kappa1, gamma1, beta) + deldellapG(G, gamma1,kappa1,alpha,beta)))


## higher matrix elements
def G1s3t(G, alpha, beta):
    return -b**2/5*lapG(G, alpha,beta)


# symmetry for matrix elements of form l sigma, 3s
def g13sSym(G, alpha,kappa2,kappa1,beta):
    return (deldelG(G, alpha,beta,kappa1,kappa2) + deldelG(G, alpha,kappa2,kappa1,beta) + deldelG(G, alpha,kappa1,beta,kappa2))/3 - (
        KroneckerDelta[kappa1,kappa2]*lapG(G, alpha,beta) + KroneckerDelta[kappa1,beta]*lapG(G, alpha,kappa2) + KroneckerDelta[beta,kappa2]*lapG(G, alpha,kappa1))/15

def G1s3s(G, alpha,kappa2, kappa1, beta):
    return b**2*(g13sSym(G, alpha,kappa2,kappa1,beta) + 5*b**2/21*Laplacian(g13sSym(G, alpha,kappa2,kappa1,beta)))


# symmetry of matrix elements of form 3s, l sigma
def g3s1Sym(G, alpha,gamma1,gamma2,beta):
    return (deldelG(G, alpha,beta,gamma1,gamma2) + deldelG(G, gamma1,beta,gamma2,alpha) + deldelG(G, gamma2,beta,alpha,gamma1))/3 - (
        KroneckerDelta[gamma1, gamma2]*lapG(G, alpha,beta) + KroneckerDelta[gamma2,alpha]*lapG(G, gamma1,beta) + KroneckerDelta[alpha,gamma1]*lapG(G, gamma2,beta))/15

def G3s1s(G, alpha,gamma1,gamma2,beta):
    return b**2*(g3s1Sym(G, alpha,gamma1,gamma2,beta) + 5*b**2/21*Laplacian(g3s1Sym(G, alpha,gamma1,gamma2,beta)))

def G3a1s(G, Lambda, kappa2, beta):
    return b**2/2*(diff(Curl(G, Lambda, beta), rij[kappa2]) + diff(Curl(G, kappa2,beta), rij[Lambda]))

def G1s3a(G, alpha, nu2, sigma):
    return 2*b**2/3*(diff(tilde_Curl(G, alpha,nu2),rij[sigma]) - SumG(G, nu2,sigma,alpha)/3) # sign change due to --> eps(sigma,nu1,beta)


def g3sCurlSym(G, alpha,gamma1,gamma2,mu):
    return (diff(tilde_Curl(G, alpha,mu), rij[gamma1],rij[gamma2]) + diff(tilde_Curl(G, gamma1,mu), rij[gamma2],rij[alpha]) + diff(tilde_Curl(G, gamma2,mu), rij[alpha],rij[gamma1]))/3
        
def G3s2a(G, alpha,gamma1,gamma2,mu):
    return -b**3/2*g3sCurlSym(G, alpha, gamma1, gamma2, mu)## sign change due to -> eps(mu, kappa1, beta)

def G3a2a(G, Lambda, kappa2, mu):
    return -b**3/4*(diff(CurlCurl(G, Lambda,mu), rij[kappa2]) + diff(CurlCurl(G, kappa2,mu), rij[Lambda])) #sign change -> eps(mu,kappa1,beta)

def G2s3t(G, alpha,gamma1,nu2):
    return -b**3/10*(dellapG(G, alpha,nu2,gamma1) + dellapG(G, gamma1,nu2,alpha))

def G3s3t(G, alpha,gamma1,gamma2,nu2):
    return -b**4/5*Laplacian(g3s1Sym(G, alpha,gamma1,gamma2,nu2))

def G2s3a(G, alpha,gamma1,mu,nu2): #sign change -> eps(mu,nu1,beta)
    return b**3/3*( diff(tilde_Curl(G, alpha,mu), rij[gamma1], rij[nu2]) - diff(SumG(G, mu,nu2,alpha), rij[gamma1])/3
                   + diff(tilde_Curl(G, gamma1,mu), rij[alpha], rij[nu2]) - diff(SumG(G, mu,nu2,gamma1), rij[alpha])/3)

def G3a3a(G, Lambda, kappa2, mu, nu2): #sign change -> eps(mu,nu1,beta)
    return b**4/3*(diff(CurlCurl(G, Lambda,mu), rij[nu2], rij[kappa2]) + diff(CurlCurl(G, kappa2,mu), rij[nu2], rij[Lambda]))

def G3s3a(G, alpha,gamma1,gamma2,mu,nu2): #sign change -> eps(mu,nu1,beta)
    return 2*b**4/3*(diff(g3sCurlSym(G, alpha,gamma1,gamma2,mu), rij[nu2]) - Laplacian(SumG3s1Sym(G, mu,nu2,alpha,gamma1,gamma2))/3)


def g2s3sSym(G, alpha,gamma1,kappa2, kappa1, beta):
    return diff(g13sSym(G, alpha,kappa2,kappa1,beta), rij[gamma1]) + diff(g13sSym(G, gamma1, kappa2, kappa1, beta), rij[alpha])

def G2s3s(G, alpha,gamma1,kappa2, kappa1, beta):
    return b**3/2*(g2s3sSym(G, alpha,gamma1,kappa2, kappa1, beta) + 6*b**2/35*Laplacian(g2s3sSym(G, alpha,gamma1,kappa2, kappa1, beta)))


def gCurl3sSym(G, tau, kappa2, kappa1, beta):
    return (diff(Curl(G, tau, beta), rij[kappa1],rij[kappa2]) + diff(Curl(G, tau,kappa1), rij[kappa2],rij[beta]) + diff(Curl(G, tau,kappa2), rij[beta],rij[kappa1]))/3 

def G3a3s(G, Lambda, eta2, kappa2, kappa1, beta):
    return b**4/2*(diff(gCurl3sSym(G, Lambda, kappa1, kappa2, beta), rij[eta2]) + diff(gCurl3sSym(G, eta2, kappa1, kappa2, beta), rij[Lambda]))


def deldelg13sSym(G, alpha,gamma1,gamma2,kappa2,kappa1,beta):
    return diff(g13sSym(G, alpha,kappa2,kappa1,beta), rij[gamma1], rij[gamma2])

def lapg13sSym(G, alpha,kappa2,kappa1,beta):
    return Laplacian(g13sSym(G, alpha,kappa2,kappa1,beta))

def g3s3sSym(G, alpha,gamma1,gamma2,kappa2,kappa1,beta):
    return (deldelg13sSym(G, alpha,gamma1,gamma2,kappa2,kappa1,beta) + deldelg13sSym(G, gamma1,gamma2,alpha,kappa2,kappa1,beta) + 
                deldelg13sSym(G, gamma2,alpha,gamma1,kappa2,kappa1,beta))/3 - (
                KroneckerDelta[gamma1,gamma2]*lapg13sSym(G, alpha, kappa2, kappa1, beta) + 
                KroneckerDelta[gamma2,alpha]*lapg13sSym(G, gamma1, kappa2, kappa1, beta) + 
                KroneckerDelta[alpha,gamma1]*lapg13sSym(G, gamma2, kappa2, kappa1, beta))/15

def G3s3s(G, alpha,gamma1,gamma2,kappa2,kappa1,beta):
    return b**4*(g3s3sSym(G, alpha,gamma1,gamma2,kappa2,kappa1,beta) + b**2/7*Laplacian(g3s3sSym(G, alpha,gamma1,gamma2,kappa2,kappa1,beta)))

def G3a2s(G, Lambda, kappa2, kappa1, beta):
    return -b**3/4*(diff(Curl(G, Lambda,beta), rij[kappa1], rij[kappa2]) + diff(Curl(G, Lambda,kappa1), rij[beta], rij[kappa2])
                       + diff(Curl(G, kappa2,beta), rij[kappa1], rij[Lambda]) + diff(Curl(G, kappa2,kappa1), rij[beta], rij[Lambda]))


def g3s2sSym(G, alpha,gamma1,gamma2, kappa1, beta):
    return diff(g3s1Sym(G, alpha,gamma1,gamma2,beta), rij[kappa1]) + diff(g3s1Sym(G, alpha,gamma1,gamma2,kappa1), rij[beta])

def G3s2s(G, alpha,gamma1,gamma2,kappa1,beta):
    return -b**3/2*(g3s2sSym(G, alpha,gamma1,gamma2, kappa1, beta) + 6*b**2/35*Laplacian(g3s2sSym(G, alpha,gamma1,gamma2, kappa1, beta)))

def G2a3a(G, Lambda, mu, nu2): # sign change -> eps(mu,nu1,beta)
    return 2*b**3/3*diff(CurlCurl(G, Lambda, mu), rij[nu2])

def G2a3s(G, Lambda,kappa2,kappa1,beta):
    return b**3*gCurl3sSym(G, Lambda, kappa1, kappa2, beta)

In [199]:
## test functional structure of G3s3s, then symmetry - still bad condition number? 
def testG3s3s(G, alpha, gamma1, gamma2, kappa2, kappa1, beta):
    return deldeldeldelG(G, alpha, beta, gamma1, gamma2, kappa1, kappa2)

In [200]:
n_jobs = 50

In [201]:
with parallel_backend('multiprocessing'):
    listtestG3s3s = Parallel(n_jobs=n_jobs)(
        delayed(simpleG)(testG3s3s, 0, GF, ki, li, mi, kj, lj, mj) for ki in range(2) for li in range(ki,2) for mi in range(li,3)
                                                                    for kj in range(2) for lj in range(kj,2) for mj in range(lj,3))
    mattestG3s3s = 1/L**3 * Matrix(listtestG3s3s).reshape(7,7)
    print('testG3s3s done')

testG3s3s done


In [202]:
sympy.simplify(mattestG3s3s[0,1])

4*pi**2*(-81*L**4*xi**4*exp(4*pi**2/(L**2*xi**2)) - 128*L**4*xi**4*exp(3*pi**2/(L**2*xi**2)) - 36*L**4*xi**4 - 648*pi**2*L**2*xi**2*exp(4*pi**2/(L**2*xi**2)) - 1152*pi**2*L**2*xi**2*exp(3*pi**2/(L**2*xi**2)) - 432*pi**2*L**2*xi**2 - 10368*pi**4*exp(4*pi**2/(L**2*xi**2)) - 20736*pi**4*exp(3*pi**2/(L**2*xi**2)) - 10368*pi**4)*exp(-12*pi**2/(L**2*xi**2))/(81*L**9*eta*xi**4)

In [203]:
ME_file = "testG3s3s.py"
dimH=7

with open(ME_file, "w") as text_file:
    print("import numpy", file=text_file)
    print("from math import *", file=text_file)
    print("PI = 3.14159265359\n", file=text_file)
    
    print("def testG3s3s(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(mattestG3s3s),(dimH,dimH)).tolist()), file=text_file)
    

## replace things for easier usage with python     
with open(ME_file, 'r') as file:
    filedata = file.read()
    
filedata = filedata.replace('pi', 'PI')
filedata = filedata.replace('sqrt', 'numpy.sqrt')

with open(ME_file, 'w') as file:
    file.write(filedata)

In [198]:
sympy.solve(mattestG3s3s[5,5].subs({xi:sqrt(pi)/L}), L)

[]

In [188]:
bla = sympy.solve(G3a3a(GF, 0,0,0 ,0).subs({xij:L, yij:L, zij:0, xi:sqrt(pi)/L, b:1}), L)

In [189]:
bla

[]

In [190]:
bla[1].evalf()

IndexError: list index out of range

In [135]:
bla = sympy.solve(Laplacian(GF(0, 0)).subs({xij:0, yij:0, zij:0, xi:sqrt(pi)/L, b:1}), L)

In [206]:
sympy.simplify(mattestG3s3s[0,1] - mattestG3s3s[1,0])

8*pi**2*(-9*L**4*xi**4*exp(4*pi**2/(L**2*xi**2)) - 16*L**4*xi**4*exp(3*pi**2/(L**2*xi**2)) - 6*L**4*xi**4 - 72*pi**2*L**2*xi**2*exp(4*pi**2/(L**2*xi**2)) - 144*pi**2*L**2*xi**2*exp(3*pi**2/(L**2*xi**2)) - 72*pi**2*L**2*xi**2 - 1152*pi**4*exp(4*pi**2/(L**2*xi**2)) - 2592*pi**4*exp(3*pi**2/(L**2*xi**2)) - 1728*pi**4)*exp(-12*pi**2/(L**2*xi**2))/(9*L**9*eta*xi**4)

In [207]:
## zero elements
def G3t1s(G, alpha,beta):
    return S(0)

def G3t2a(G, alpha,beta):
    return S(0)

def G3t2s(G, alpha, beta, kappa1):
    return S(0)

def G3t3t(G, alpha,beta):
    return S(0)

def G3t3a(G, alpha, beta, kappa1):
    return S(0)

def G3t3s(G, alpha, beta, kappa1, kappa2):
    return S(0)

def G2a3t(G, alpha,beta):
    return S(0)

def G3a3t(G, alpha, gamma1, beta):
    return S(0)

## Matrix assembly

In [208]:
## simplify elements in parallel
## subs: substitute corresponding replacement for xij
## simplify: otherwise expressions too large, eg GR3s3s is 70 MB file
## .expand(power_exp=True): otherwise large exponentials --> infty when eval numerically
## powdenest: otherwise large powers of L --> infty when eval numerically 
def simpleG(G, rpl, *args): ## which matrix element, replacement (0 or L), arguments of matrix element
    return sympy.powdenest(sympy.simplify(G(*args).subs({xij:rpl, yij:rpl, zij:rpl})).expand(power_exp=True), force=True) 

In [209]:
n_jobs = 50  ## symEngine only parallelises with multiprocessing backend

In [210]:
modes = ['1s', '2a', '2s', '3t', '3a', '3s']
modesV = ['2s', '3t'] ##possible slip modes, for double-layer matrix elements
SLDL  = ['G']#, 'K']
dim   = {'1s':3, '2a':3, '2s':5, '3t':3, '3a':5, '3s':7}

### 1) Fourier part of Green's function

In [211]:
%%time
sldl = 'G' # single-layer
GRF = 'GF' # Fourier space part of Green's function
rpl = 0    # what are xij, yij, zij replaced with? rpl = replacement
with parallel_backend('multiprocessing'):
    for mode1 in modes:
        for mode2 in modes:
            if mode1 in ['1s', '2a']:
                if mode2 in ['1s', '2a', '3t']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simpleG)(globals()[sldl+mode1+mode2], rpl, globals()[GRF], ki, kj) for ki in range(3) 
                                                                                                        for kj in range(3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:                ## GLH needs symmetry factors
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simpleG)(globals()[sldl+mode1+mode2], rpl, globals()[GRF], ki, kj, lj) for ki in range(3)
                                                                                                            for kj in range(2)
                                                                                                                for lj in range(kj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simpleG)(globals()[sldl+mode1+mode2], rpl, globals()[GRF], ki, kj, lj, mj) for ki in range(3)
                                                                                                                for kj in range(2)
                                                                                                                    for lj in range(kj,2)
                                                                                                                        for mj in range(lj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(sldl+mode1+mode2+' done')



            elif mode1 == '3t':
                if mode2 in ['1s', '2a', '3t']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simpleG)(globals()[sldl+mode1+mode2], rpl, globals()[GRF], ki, kj) for ki in range(3) 
                                                                                                        for kj in range(3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simpleG)(globals()[sldl+mode1+mode2], rpl, globals()[GRF], ki, kj, lj) for ki in range(3)
                                                                                                            for kj in range(2)
                                                                                                                for lj in range(kj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simpleG)(globals()[sldl+mode1+mode2], rpl, globals()[GRF], ki, kj, lj, mj) for ki in range(3)
                                                                                                                for kj in range(2)
                                                                                                                    for lj in range(kj,2)
                                                                                                                        for mj in range(lj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(sldl+mode1+mode2+' done')



            elif mode1 in ['2s', '3a']:
                if mode2 in ['1s', '2a', '3t']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simpleG)(globals()[sldl+mode1+mode2], rpl, globals()[GRF], ki, li, kj) for ki in range(2) 
                                                                                                            for li in range(ki,3)
                                                                                                                for kj in range(3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simpleG)(globals()[sldl+mode1+mode2], rpl, globals()[GRF], ki, li, kj, lj) for ki in range(2)
                                                                                                                for li in range(ki,3)
                                                                                                                    for kj in range(2)
                                                                                                                        for lj in range(kj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simpleG)(globals()[sldl+mode1+mode2], rpl, globals()[GRF], ki, li, kj, lj, mj) for ki in range(2)
                                                                                                                    for li in range(ki,3)
                                                                                                                        for kj in range(2)
                                                                                                                            for lj in range(kj,2)
                                                                                                                                for mj in range(lj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(sldl+mode1+mode2+' done')



            elif mode1 == '3s':
                if mode2 in ['1s', '2a', '3t']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simpleG)(globals()[sldl+mode1+mode2], rpl, globals()[GRF], ki, li, mi, kj) for ki in range(2) 
                                                                                                                for li in range(ki,2)
                                                                                                                    for mi in range(li,3)
                                                                                                                        for kj in range(3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(sldl+mode1+mode2+' done')

                elif mode2 in ['2s', '3a']:
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simpleG)(globals()[sldl+mode1+mode2], rpl, globals()[GRF], ki, li, mi, kj, lj) for ki in range(2)
                                                                                                                    for li in range(ki,2)
                                                                                                                        for mi in range(li,3)
                                                                                                                            for kj in range(2)
                                                                                                                                for lj in range(kj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(sldl+mode1+mode2+' done')

                elif mode2 =='3s':
                    globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                        delayed(simpleG)(globals()[sldl+mode1+mode2], rpl, globals()[GRF], ki, li, mi, kj, lj, mj) for ki in range(2)
                                                                                                                        for li in range(ki,2)
                                                                                                                            for mi in range(li,3)
                                                                                                                                for kj in range(2)
                                                                                                                                    for lj in range(kj,2)
                                                                                                                                        for mj in range(lj,3))
                    globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                    print(sldl+mode1+mode2+' done')

G1s1s done
G1s2a done
G1s2s done
G1s3t done
G1s3a done
G1s3s done
G2a1s done
G2a2a done
G2a2s done
G2a3t done
G2a3a done
G2a3s done
G2s1s done
G2s2a done
G2s2s done
G2s3t done
G2s3a done
G2s3s done
G3t1s done
G3t2a done
G3t2s done
G3t3t done
G3t3a done
G3t3s done
G3a1s done
G3a2a done
G3a2s done
G3a3t done
G3a3a done
G3a3s done
G3s1s done
G3s2a done
G3s2s done
G3s3t done
G3s3a done
G3s3s done
CPU times: user 17.6 s, sys: 14 s, total: 31.6 s
Wall time: 1min 23s


### 3) add Green's functions

In [212]:
for mode1 in modes:
    for mode2 in modes:
        #globals()['mat'+sldl+mode1+mode2] = 1/(8*pi*eta) * globals()['matGR'+mode1+mode2] + 1/L**3 * globals()['matGF'+mode1+mode2]
        globals()['mat'+sldl+mode1+mode2] = 1/L**3 * globals()['matGF'+mode1+mode2] ## Fourier space Green's function only
        print(sldl+mode1+mode2+' done')

G1s1s done
G1s2a done
G1s2s done
G1s3t done
G1s3a done
G1s3s done
G2a1s done
G2a2a done
G2a2s done
G2a3t done
G2a3a done
G2a3s done
G2s1s done
G2s2a done
G2s2s done
G2s3t done
G2s3a done
G2s3s done
G3t1s done
G3t2a done
G3t2s done
G3t3t done
G3t3a done
G3t3s done
G3a1s done
G3a2a done
G3a2s done
G3a3t done
G3a3a done
G3a3s done
G3s1s done
G3s2a done
G3s2s done
G3s3t done
G3s3a done
G3s3s done


### Create double-layer matrices

In [213]:
for mode1 in modes:
    for mode2 in modesV:
        if mode2 == '2s':
            globals()['matK'+mode1+mode2] = 8*pi*eta*b/3 * globals()['matG'+mode1+mode2]
            print('K'+mode1+mode2+' done')
            
        elif mode2 == '3t':
            globals()['matK'+mode1+mode2] = 6*pi*eta*b/5 * globals()['matG'+mode1+mode2]
            print('K'+mode1+mode2+' done')

K1s2s done
K1s3t done
K2a2s done
K2a3t done
K2s2s done
K2s3t done
K3t2s done
K3t3t done
K3a2s done
K3a3t done
K3s2s done
K3s3t done


In [214]:
## save all G's and K's to files, and re-load into a different sympy file for simplification over parallel modes ? Nope
## does not need to be fast at the moment, just save them all as usual and use for one time-step

In [215]:
#text_file = open("matrixElements/G3s3s.txt", "w")
#nn = text_file.write(str(matG3s3s))
#text_file.close()
#nn ##that's a 70MB file lol

### Delta tensors

In [216]:
## Delta tensors as 'identity' matrices
def Delta1(alpha, beta):
    return KroneckerDelta[alpha,beta]

def Delta2(alpha, gamma1, beta, kappa1):
    return (KroneckerDelta[alpha,beta]*KroneckerDelta[gamma1,kappa1]/2
            + KroneckerDelta[alpha,kappa1]*KroneckerDelta[gamma1,beta]/2
            -KroneckerDelta[alpha,gamma1]*KroneckerDelta[beta,kappa1]/3)

def Delta3(alpha, gamma1, gamma2, beta, kappa1, kappa2):
    return (KroneckerDelta[alpha,beta]*KroneckerDelta[gamma1,kappa1]*KroneckerDelta[gamma2,kappa2] 
                + KroneckerDelta[alpha,beta]*KroneckerDelta[gamma1,kappa2]*KroneckerDelta[gamma2,kappa1] 
                + KroneckerDelta[alpha,kappa1]*KroneckerDelta[gamma1,beta]*KroneckerDelta[gamma2,kappa2] 
                + KroneckerDelta[alpha,kappa1]*KroneckerDelta[gamma1,kappa2]*KroneckerDelta[gamma2,beta] 
                + KroneckerDelta[alpha,kappa2]*KroneckerDelta[gamma1,beta]*KroneckerDelta[gamma2,kappa1]
                + KroneckerDelta[alpha,kappa2]*KroneckerDelta[gamma1,kappa1]*KroneckerDelta[gamma2,beta])/6 - (
        (KroneckerDelta[beta,kappa1]*KroneckerDelta[gamma2,kappa2] + KroneckerDelta[beta,kappa2]*KroneckerDelta[gamma2,kappa1]
         + KroneckerDelta[kappa1,kappa2]*KroneckerDelta[gamma2,beta])*KroneckerDelta[alpha,gamma1]
        +(KroneckerDelta[beta,kappa1]*KroneckerDelta[alpha,kappa2] + KroneckerDelta[beta,kappa2]*KroneckerDelta[alpha,kappa1]
          + KroneckerDelta[kappa1,kappa2]*KroneckerDelta[alpha,beta])*KroneckerDelta[gamma1,gamma2]
        +(KroneckerDelta[beta,kappa1]*KroneckerDelta[gamma1,kappa2] + KroneckerDelta[beta,kappa2]*KroneckerDelta[gamma1,kappa1]
          + KroneckerDelta[kappa1,kappa2]*KroneckerDelta[gamma1,beta])*KroneckerDelta[alpha,gamma2])/15

In [217]:
n_jobs = 1

with parallel_backend('multiprocessing'):
    listDelta1 = Parallel(n_jobs=n_jobs)(delayed(Delta1)(ki,kj) for ki in range(3)
                                                                     for kj in range(3))
matDelta1   = sympy.Matrix(listDelta1).reshape(3,3)

##
with parallel_backend('multiprocessing'):
    listDelta2 = Parallel(n_jobs=n_jobs)(delayed(Delta2)(ki,li, kj,lj) for ki in range(2)
                                                                            for li in range(ki,3)
                                                                                for kj in range(2)
                                                                                  for lj in range(kj,3))
matDelta2   = sympy.Matrix(listDelta2).reshape(5,5)

##
with parallel_backend('multiprocessing'):
    listDelta3 = Parallel(n_jobs=n_jobs)(delayed(Delta3)(ki,li,mi, kj,lj,mj) for ki in range(2)
                                                                                for li in range(ki,2)
                                                                                    for mi in range(li,3)
                                                                                        for kj in range(2)
                                                                                            for lj in range(kj,2)
                                                                                                for mj in range(lj,3))
matDelta3   = sympy.Matrix(listDelta3).reshape(7,7)

### Vectors

In [218]:
## In python code will be force[i,0], so have to create force as np.zeros([3*Np,1]) object instead of np.zeros([3*Np])
force  = sympy.Matrix(sympy.MatrixSymbol('force',3,1)) ##create an actual Matrix from it, have to pass to 'Matrix' 
torque = sympy.Matrix(sympy.MatrixSymbol('torque',3,1))
VH     = sympy.Matrix(sympy.BlockDiagMatrix(matDelta2.inv(), matDelta1.inv(), matDelta2.inv(), matDelta3.inv()))*sympy.Matrix(sympy.MatrixSymbol('VH',20,1))
FH     = sympy.Matrix(sympy.BlockDiagMatrix(matDelta2.inv(), matDelta1.inv(), matDelta2.inv(), matDelta3.inv()))*sympy.Matrix(sympy.MatrixSymbol('FH',20,1))

In [219]:
FH

Matrix([
[    2*FH[0, 0] + FH[3, 0]],
[               2*FH[1, 0]],
[               2*FH[2, 0]],
[    FH[0, 0] + 2*FH[3, 0]],
[               2*FH[4, 0]],
[                 FH[5, 0]],
[                 FH[6, 0]],
[                 FH[7, 0]],
[   2*FH[8, 0] + FH[11, 0]],
[               2*FH[9, 0]],
[              2*FH[10, 0]],
[   FH[8, 0] + 2*FH[11, 0]],
[              2*FH[12, 0]],
[4*FH[13, 0] + 3*FH[16, 0]],
[6*FH[14, 0] + 3*FH[18, 0]],
[  4*FH[15, 0] + FH[19, 0]],
[3*FH[13, 0] + 6*FH[16, 0]],
[              6*FH[17, 0]],
[3*FH[14, 0] + 4*FH[18, 0]],
[  FH[15, 0] + 4*FH[19, 0]]])

### Create block matrices and multiply them with vectors

In [220]:
matG1sH = sympy.Matrix(sympy.BlockMatrix([sympy.Matrix(matG1s2s), sympy.Matrix(matG1s3t), sympy.Matrix(matG1s3a), sympy.Matrix(matG1s3s)]))
matG2aH = sympy.Matrix(sympy.BlockMatrix([sympy.Matrix(matG2a2s), sympy.Matrix(matG2a3t), sympy.Matrix(matG2a3a), sympy.Matrix(matG2a3s)]))

matK1sH = sympy.Matrix(sympy.BlockMatrix([sympy.Matrix(matK1s2s), sympy.Matrix(matK1s3t), sympy.Matrix(sympy.ZeroMatrix(3,12))]))
matK2aH = sympy.Matrix(sympy.BlockMatrix([sympy.Matrix(matK2a2s), sympy.Matrix(matK2a3t), sympy.Matrix(sympy.ZeroMatrix(3,12))]))

matGH1s = sympy.Matrix(sympy.BlockMatrix([[sympy.Matrix(matG2s1s)],
                              [sympy.Matrix(matG3t1s)],
                              [sympy.Matrix(matG3a1s)],
                              [sympy.Matrix(matG3s1s)]]))
matGH2a = sympy.Matrix(sympy.BlockMatrix([[sympy.Matrix(matG2s2a)],
                              [sympy.Matrix(matG3t2a)],
                              [sympy.Matrix(matG3a2a)],
                              [sympy.Matrix(matG3s2a)]]))

matGHH = sympy.Matrix(sympy.BlockMatrix([[sympy.Matrix(matG2s2s), sympy.Matrix(matG2s3t), sympy.Matrix(matG2s3a), sympy.Matrix(matG2s3s)],
                             [sympy.Matrix(matG3t2s), sympy.Matrix(matG3t3t), sympy.Matrix(matG3t3a), sympy.Matrix(matG3t3s)],
                             [sympy.Matrix(matG3a2s), sympy.Matrix(matG3a3t), sympy.Matrix(matG3a3a), sympy.Matrix(matG3a3s)],
                             [sympy.Matrix(matG3s2s), sympy.Matrix(matG3s3t), sympy.Matrix(matG3s3a), sympy.Matrix(matG3s3s)]]))

matKHH_nonzero = sympy.Matrix(sympy.BlockMatrix([[sympy.Matrix(matK2s2s), sympy.Matrix(matK2s3t)],
                                     [sympy.Matrix(matK3t2s), sympy.Matrix(matK3t3t)],
                                     [sympy.Matrix(matK3a2s), sympy.Matrix(matK3a3t)],
                                     [sympy.Matrix(matK3s2s), sympy.Matrix(matK3s3t)]]))
matKHH = sympy.Matrix(sympy.BlockMatrix([matKHH_nonzero, sympy.Matrix(sympy.ZeroMatrix(20,12))]))

# self-interaction

g2s = 3/(20*pi*eta*b)
g3t = 1/(2*pi*eta*b)
g3a = 3/(2*pi*eta*b)
g3s = 6/(7*pi*eta*b)

matGoHH = sympy.Matrix(sympy.BlockDiagMatrix(g2s*matDelta2,
                                             g3t*matDelta1,
                                             g3a*matDelta2,
                                             g3s*matDelta3))


halfMinusk2s = S(3)/5 #0.6
halfMinusk3t = S(2)/5 #0.4
halfMinusk3a = S(4)/5 #0.8
halfMinusk3s = S(19)/35

matKoHH = sympy.Matrix(sympy.BlockDiagMatrix(halfMinusk2s*matDelta2,
                                             halfMinusk3t*matDelta1,
                                             halfMinusk3a*matDelta2,
                                             halfMinusk3s*matDelta3))

In [221]:
## a) can ignore ls symmetries, multiply by (full) Delta tensors and then pick unique elements
## b) return matrices as sympy.Matrix, instead of Matrix. 

In [222]:
vecGH1sF = matGH1s*force
vecGH2aT = matGH2a*torque

vecKoHHVH = matKoHH*VH
vecKHHVH = matKHH*VH

vecGoHHFH = matGoHH*FH
vecGHHFH = matGHH*FH

vecG1s1sF = sympy.Matrix(matG1s1s)*force
vecG2a1sF = sympy.Matrix(matG2a1s)*force

vecG1s2aT = sympy.Matrix(matG1s2a)*torque
vecG2a2aT = sympy.Matrix(matG2a2a)*torque

vecG1sHFH = matG1sH*FH
vecG2aHFH = matG2aH*FH

vecK1sHVH = matK1sH*VH
vecK2aHVH = matK2aH*VH

_____________

### Write symbolic results to file as reusable python functions

In [223]:
ME_file = "periodic_"+str(Nb)+"_"+str(Nm)+"_old.py"
dimH = 20

with open(ME_file, "w") as text_file:
    print("import numpy as np", file=text_file)
    print("from math import *", file=text_file)
    print("PI = 3.14159265359\n", file=text_file)
    
    print("def G1s1sF(v, L, xi, b,eta, force):", file=text_file)
    print("    v += np.array({})".format(np.reshape(np.asarray(vecG1s1sF),(3,)).tolist()), file=text_file) ##cannot mulitply list by 0.5/b 
    print("    return\n", file=text_file)
    
    print("def G2a1sF(o, L, xi, b,eta, force):", file=text_file)
    print("    o += 0.5/b*np.array({})".format(np.reshape(np.asarray(vecG2a1sF),(3,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def G1s2aT(v, L, xi, b,eta, torque):", file=text_file)
    print("    v += 1./b*np.array({})".format(np.reshape(np.asarray(vecG1s2aT),(3,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def G2a2aT(o, L, xi, b,eta, torque):", file=text_file)
    print("    o += 0.5/(b*b)*np.array({})".format(np.reshape(np.asarray(vecG2a2aT),(3,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GH1sF(gh1sf, L, xi, b,eta, force):", file=text_file)
    print("    gh1sf[:] += {}".format(np.reshape(np.asarray(vecGH1sF),(dimH,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GH2aT(gh2at, L, xi, b,eta, torque):", file=text_file)
    print("    gh2at[:] += {}".format(np.reshape(np.asarray(vecGH2aT),(dimH,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def KoHHVH(khhvh, b,eta, VH):", file=text_file)
    print("    khhvh[:] -= {}".format(np.reshape(np.asarray(vecKoHHVH),(dimH,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def KHHVH(khhvh, L, xi, b,eta, VH):", file=text_file)
    print("    khhvh[:] += {}".format(np.reshape(np.asarray(vecKHHVH),(dimH,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GoHHFH(ghhfh, b,eta, FH):", file=text_file)
    print("    ghhfh[:] += {}".format(np.reshape(np.asarray(vecGoHHFH),(dimH,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GHHFH(ghhfh, L, xi, b,eta, FH):", file=text_file)
    print("    ghhfh[:] += {}".format(np.reshape(np.asarray(vecGHHFH),(dimH,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def G1sHFH(v, L, xi, b,eta, FH):", file=text_file)
    print("    v -= np.array({})".format(np.reshape(np.asarray(vecG1sHFH),(3,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def G2aHFH(o, L, xi, b,eta, FH):", file=text_file)
    print("    o -= 0.5/b*np.array({})".format(np.reshape(np.asarray(vecG2aHFH),(3,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def K1sHVH(v, L, xi, b,eta, VH):", file=text_file)
    print("    v += np.array({})".format(np.reshape(np.asarray(vecK1sHVH),(3,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def K2aHVH(o, L, xi, b,eta, VH):", file=text_file)
    print("    o += 0.5/b*np.array({})".format(np.reshape(np.asarray(vecK2aHVH),(3,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
## replace things for easier usage with python     
with open(ME_file, 'r') as file:
    filedata = file.read()
    
filedata = filedata.replace('pi', 'PI')
#filedata = filedata.replace('sqrt', 'numpy.sqrt')

filedata = re.sub("force\[(\d), 0\]", r"force[\1]", filedata)
filedata = re.sub("torque\[(\d), 0\]", r"torque[\1]", filedata)
filedata = re.sub("VH\[(\d||\d\d), 0\]", r"VH[\1]", filedata)
filedata = re.sub("FH\[(\d||\d\d), 0\]", r"FH[\1]", filedata)

with open(ME_file, 'w') as file:
    file.write(filedata)

______________________________________
_______________________

____
___

### Symbolic matrices to file for direct solver

In [224]:
ME_file = "matrixPeriodic_"+str(Nb)+"_"+str(Nm)+"_old.py"

with open(ME_file, "w") as text_file:
    print("import numpy", file=text_file)
    print("from math import *", file=text_file)
    print("PI = 3.14159265359\n", file=text_file)
    
    print("def G1s1s(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG1s1s),(3,3)).tolist()), file=text_file)
    print("def G2a1s(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG2a1s),(3,3)).tolist()), file=text_file)
    
    print("def G1s2a(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG1s2a),(3,3)).tolist()), file=text_file)
    print("def G2a2a(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG2a2a),(3,3)).tolist()), file=text_file)
    
    print("def GH1s(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matGH1s),(dimH,3)).tolist()), file=text_file)
    print("def GH2a(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matGH2a),(dimH,3)).tolist()), file=text_file)
    
    print("def KoHH(b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matKoHH),(dimH,dimH)).tolist()), file=text_file)
    print("def KHH(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matKHH),(dimH,dimH)).tolist()), file=text_file)
    
    print("def GoHH(b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matGoHH),(dimH,dimH)).tolist()), file=text_file)
    print("def GHH(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matGHH),(dimH,dimH)).tolist()), file=text_file)
    
    print("def G1sH(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG1sH),(3,dimH)).tolist()), file=text_file)
    print("def G2aH(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG2aH),(3,dimH)).tolist()), file=text_file)
    
    print("def K1sH(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matK1sH),(3,dimH)).tolist()), file=text_file)
    print("def K2aH(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matK2aH),(3,dimH)).tolist()), file=text_file)
    
    
    
## replace things for easier usage with python     
with open(ME_file, 'r') as file:
    filedata = file.read()
    
filedata = filedata.replace('pi', 'PI')
filedata = filedata.replace('sqrt', 'numpy.sqrt')

with open(ME_file, 'w') as file:
    file.write(filedata)

____________
___________


In [225]:
matGF1s2s

[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]

In [226]:
##delete all but 3s, which is what causes the pole

In [227]:
matG1sH = sympy.Matrix(sympy.BlockMatrix([sympy.Matrix(matG1s3s)]))
matG2aH = sympy.Matrix(sympy.BlockMatrix([sympy.Matrix(matG2a3s)]))

matK1sH = sympy.Matrix(sympy.BlockMatrix([sympy.Matrix(sympy.ZeroMatrix(3,7))]))
matK2aH = sympy.Matrix(sympy.BlockMatrix([sympy.Matrix(sympy.ZeroMatrix(3,7))]))

matGH1s = sympy.Matrix(sympy.BlockMatrix([[sympy.Matrix(matG3s1s)]]))
matGH2a = sympy.Matrix(sympy.BlockMatrix([[sympy.Matrix(matG3s2a)]]))

matGHH = sympy.Matrix(sympy.BlockMatrix([[sympy.Matrix(matG3s3s)]]))

matKHH_nonzero = sympy.Matrix(sympy.BlockMatrix([[sympy.Matrix(sympy.ZeroMatrix(7,7))]]))
matKHH = sympy.Matrix(sympy.BlockMatrix([matKHH_nonzero]))

# self-interaction

g3s = 6/(7*pi*eta*b)

matGoHH = sympy.Matrix(sympy.BlockDiagMatrix(g3s*matDelta3))



halfMinusk3s = S(19)/35

matKoHH = sympy.Matrix(sympy.BlockDiagMatrix(halfMinusk3s*matDelta3))

In [228]:
ME_file = "matrixPeriodic_"+str(Nb)+"_"+str(Nm)+"_3s.py"
dimH=7

with open(ME_file, "w") as text_file:
    print("import numpy", file=text_file)
    print("from math import *", file=text_file)
    print("PI = 3.14159265359\n", file=text_file)
    
    print("def G1s1s(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG1s1s),(3,3)).tolist()), file=text_file)
    print("def G2a1s(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG2a1s),(3,3)).tolist()), file=text_file)
    
    print("def G1s2a(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG1s2a),(3,3)).tolist()), file=text_file)
    print("def G2a2a(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG2a2a),(3,3)).tolist()), file=text_file)
    
    print("def GH1s(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matGH1s),(dimH,3)).tolist()), file=text_file)
    print("def GH2a(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matGH2a),(dimH,3)).tolist()), file=text_file)
    
    print("def KoHH(b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matKoHH),(dimH,dimH)).tolist()), file=text_file)
    print("def KHH(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matKHH),(dimH,dimH)).tolist()), file=text_file)
    
    print("def GoHH(b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matGoHH),(dimH,dimH)).tolist()), file=text_file)
    print("def GHH(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matGHH),(dimH,dimH)).tolist()), file=text_file)
    
    print("def G1sH(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG1sH),(3,dimH)).tolist()), file=text_file)
    print("def G2aH(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG2aH),(3,dimH)).tolist()), file=text_file)
    
    print("def K1sH(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matK1sH),(3,dimH)).tolist()), file=text_file)
    print("def K2aH(L, xi, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matK2aH),(3,dimH)).tolist()), file=text_file)
    
    
    
## replace things for easier usage with python     
with open(ME_file, 'r') as file:
    filedata = file.read()
    
filedata = filedata.replace('pi', 'PI')
filedata = filedata.replace('sqrt', 'numpy.sqrt')

with open(ME_file, 'w') as file:
    file.write(filedata)

In [232]:
matG2s2s[0,2] - matG2s2s[2,0]

0

In [236]:
sympy.simplify(G2s2s(GF, 0,0, 0,0).subs({xij:0, yij:0, zij:0}))

b**2*(2700*L**2*(L**4*xi**4 + 2*pi**2*L**2*xi**2 + 8*pi**4)*exp(10*pi**2/(L**2*xi**2)) + 2400*L**2*(L**4*xi**4 + 3*pi**2*L**2*xi**2 + 18*pi**4)*exp(9*pi**2/(L**2*xi**2)) + 1728*L**2*(L**4*xi**4 + 5*pi**2*L**2*xi**2 + 50*pi**4)*exp(7*pi**2/(L**2*xi**2)) + 2700*L**2*(L**4*xi**4 + 6*pi**2*L**2*xi**2 + 72*pi**4)*exp(6*pi**2/(L**2*xi**2)) + 675*L**2*(L**4*xi**4 + 8*pi**2*L**2*xi**2 + 128*pi**4)*exp(4*pi**2/(L**2*xi**2)) + 1600*L**2*(L**4*xi**4 + 9*pi**2*L**2*xi**2 + 162*pi**4)*exp(3*pi**2/(L**2*xi**2)) + 300*L**2*(L**4*xi**4 + 12*pi**2*L**2*xi**2 + 288*pi**4) - 288*pi**2*b**2*(10*L**4*xi**4 + 120*pi**2*L**2*xi**2 + 15*(L**4*xi**4 + 2*pi**2*L**2*xi**2 + 8*pi**4)*exp(10*pi**2/(L**2*xi**2)) + 20*(L**4*xi**4 + 3*pi**2*L**2*xi**2 + 18*pi**4)*exp(9*pi**2/(L**2*xi**2)) + 24*(L**4*xi**4 + 5*pi**2*L**2*xi**2 + 50*pi**4)*exp(7*pi**2/(L**2*xi**2)) + 45*(L**4*xi**4 + 6*pi**2*L**2*xi**2 + 72*pi**4)*exp(6*pi**2/(L**2*xi**2)) + 15*(L**4*xi**4 + 8*pi**2*L**2*xi**2 + 128*pi**4)*exp(4*pi**2/(L**2*xi**2)) + 4

In [237]:
sympy.simplify(G2s2s(GF, 0,0, 0,0).subs({xij:0, yij:0, zij:0})).expand(power_exp=True)

2*b**2*exp(-2*pi**2/(L**2*xi**2))/eta + 16*b**2*exp(-3*pi**2/(L**2*xi**2))/(9*eta) + 32*b**2*exp(-5*pi**2/(L**2*xi**2))/(25*eta) + 2*b**2*exp(-6*pi**2/(L**2*xi**2))/eta + b**2*exp(-8*pi**2/(L**2*xi**2))/(2*eta) + 32*b**2*exp(-9*pi**2/(L**2*xi**2))/(27*eta) + 2*b**2*exp(-12*pi**2/(L**2*xi**2))/(9*eta) - 16*pi**2*b**4*exp(-2*pi**2/(L**2*xi**2))/(5*L**2*eta) - 64*pi**2*b**4*exp(-3*pi**2/(L**2*xi**2))/(15*L**2*eta) - 128*pi**2*b**4*exp(-5*pi**2/(L**2*xi**2))/(25*L**2*eta) - 48*pi**2*b**4*exp(-6*pi**2/(L**2*xi**2))/(5*L**2*eta) - 16*pi**2*b**4*exp(-8*pi**2/(L**2*xi**2))/(5*L**2*eta) - 128*pi**2*b**4*exp(-9*pi**2/(L**2*xi**2))/(15*L**2*eta) - 32*pi**2*b**4*exp(-12*pi**2/(L**2*xi**2))/(15*L**2*eta) + 4*pi**2*b**2*exp(-2*pi**2/(L**2*xi**2))/(L**2*eta*xi**2) + 16*pi**2*b**2*exp(-3*pi**2/(L**2*xi**2))/(3*L**2*eta*xi**2) + 32*pi**2*b**2*exp(-5*pi**2/(L**2*xi**2))/(5*L**2*eta*xi**2) + 12*pi**2*b**2*exp(-6*pi**2/(L**2*xi**2))/(L**2*eta*xi**2) + 4*pi**2*b**2*exp(-8*pi**2/(L**2*xi**2))/(L**2*eta*xi**

In [242]:
sympy.powdenest(sympy.simplify(G2s2s(GF, 0,1, 0,1).subs({xij:0, yij:0, zij:0})).expand(power_exp=True), force=True)

b**2*exp(-pi**2/(L**2*xi**2))/eta + b**2*exp(-2*pi**2/(L**2*xi**2))/eta + 4*b**2*exp(-3*pi**2/(L**2*xi**2))/(9*eta) + b**2*exp(-4*pi**2/(L**2*xi**2))/(2*eta) + 34*b**2*exp(-5*pi**2/(L**2*xi**2))/(25*eta) + b**2*exp(-6*pi**2/(L**2*xi**2))/eta + b**2*exp(-8*pi**2/(L**2*xi**2))/(4*eta) + 11*b**2*exp(-9*pi**2/(L**2*xi**2))/(27*eta) + b**2*exp(-12*pi**2/(L**2*xi**2))/(18*eta) - 4*pi**2*b**4*exp(-pi**2/(L**2*xi**2))/(5*L**2*eta) - 8*pi**2*b**4*exp(-2*pi**2/(L**2*xi**2))/(5*L**2*eta) - 16*pi**2*b**4*exp(-3*pi**2/(L**2*xi**2))/(15*L**2*eta) - 8*pi**2*b**4*exp(-4*pi**2/(L**2*xi**2))/(5*L**2*eta) - 136*pi**2*b**4*exp(-5*pi**2/(L**2*xi**2))/(25*L**2*eta) - 24*pi**2*b**4*exp(-6*pi**2/(L**2*xi**2))/(5*L**2*eta) - 8*pi**2*b**4*exp(-8*pi**2/(L**2*xi**2))/(5*L**2*eta) - 44*pi**2*b**4*exp(-9*pi**2/(L**2*xi**2))/(15*L**2*eta) - 8*pi**2*b**4*exp(-12*pi**2/(L**2*xi**2))/(15*L**2*eta) + pi**2*b**2*exp(-pi**2/(L**2*xi**2))/(L**2*eta*xi**2) + 2*pi**2*b**2*exp(-2*pi**2/(L**2*xi**2))/(L**2*eta*xi**2) + 4*pi**2

In [243]:
sympy.simplify(matG2s2s[1,1]).expand(power_exp=True)

b**2*exp(-pi**2/(L**2*xi**2))/(L**3*eta) + b**2*exp(-2*pi**2/(L**2*xi**2))/(L**3*eta) + 4*b**2*exp(-3*pi**2/(L**2*xi**2))/(9*L**3*eta) + b**2*exp(-4*pi**2/(L**2*xi**2))/(2*L**3*eta) + 34*b**2*exp(-5*pi**2/(L**2*xi**2))/(25*L**3*eta) + b**2*exp(-6*pi**2/(L**2*xi**2))/(L**3*eta) + b**2*exp(-8*pi**2/(L**2*xi**2))/(4*L**3*eta) + 11*b**2*exp(-9*pi**2/(L**2*xi**2))/(27*L**3*eta) + b**2*exp(-12*pi**2/(L**2*xi**2))/(18*L**3*eta) - 4*pi**2*b**4*exp(-pi**2/(L**2*xi**2))/(5*L**5*eta) - 8*pi**2*b**4*exp(-2*pi**2/(L**2*xi**2))/(5*L**5*eta) - 16*pi**2*b**4*exp(-3*pi**2/(L**2*xi**2))/(15*L**5*eta) - 8*pi**2*b**4*exp(-4*pi**2/(L**2*xi**2))/(5*L**5*eta) - 136*pi**2*b**4*exp(-5*pi**2/(L**2*xi**2))/(25*L**5*eta) - 24*pi**2*b**4*exp(-6*pi**2/(L**2*xi**2))/(5*L**5*eta) - 8*pi**2*b**4*exp(-8*pi**2/(L**2*xi**2))/(5*L**5*eta) - 44*pi**2*b**4*exp(-9*pi**2/(L**2*xi**2))/(15*L**5*eta) - 8*pi**2*b**4*exp(-12*pi**2/(L**2*xi**2))/(15*L**5*eta) + pi**2*b**2*exp(-pi**2/(L**2*xi**2))/(L**5*eta*xi**2) + 2*pi**2*b**2*ex

In [33]:
matG1s1s.shape

(3, 3)

In [390]:
sympy.solve(matGF3s3s[4,4].subs({b:1, eta:1/6, xi:sqrt(pi)/L}), L) ## this does not seem to tell me much tbh. GR3s3s eg also has a root wrt L

[-3.36052539015438, 3.36052539015438]

In [401]:
sympy.solve(sympy.Matrix(matGF3s1s)[0,0].subs(xi,sqrt(pi)/L), L)

[-2.88813589883052*b, 2.88813589883052*b]

In [410]:
Laplacian(Laplacian( GreenReal(0,0) )).subs({b:1, eta:1, L:10, xi:sqrt(pi)/10, xij:10, yij:10, zij:10}).evalf()

-0.0108895285293013

In [411]:
Laplacian(Laplacian( GreenFourier(0,0) )).subs({b:1, eta:1, L:10, xi:sqrt(pi)/10, xij:0, yij:0, zij:0}).evalf()

2.7477148249433

In [413]:
(diff(GreenFourier(0,0), rij[0]) + diff(GreenFourier(1,0), rij[1]) + diff(GreenFourier(2,0), rij[2])).subs({b:1, eta:1, L:10, xi:sqrt(pi)/10, xij:10, yij:10, zij:10}).evalf()

0.0

In [415]:
Laplacian(Laplacian( GreenFourier(0,0) )).subs(
    {b:1, eta:1, L:10, xi:sqrt(pi)/10, xij:0, yij:0, zij:0}).evalf()

2.7477148249433

In [438]:
k_vec = Matrix([kx, ky, kz])
norm(k_vec)

sqrt(kx**2 + ky**2 + kz**2)

In [448]:
def M1r(alpha, beta):
    func = norm(rij)*erfc(xi*norm(rij))
    return KroneckerDelta[alpha, beta]*Laplacian(func) - diff( func, rij[alpha], rij[beta])

def M2r(alpha, beta):
    func = norm(rij)*erf(xi*norm(rij))
    return KroneckerDelta[alpha, beta]*Laplacian(func) - diff( func, rij[alpha], rij[beta])

def Mr(alpha, beta):
    return M1r(alpha,beta) + M2r(alpha,beta)

In [452]:
Laplacian(Laplacian( Mr(2,2))).subs({xij:1, yij:1, zij:1, xi:1})

0

In [465]:
M2r(0,1).subs({xij:0, yij:0, zij:0})

-(nan + nan*xi/sqrt(pi))

In [466]:
M2r(0,1)

-(-xij*yij*erf(xi*sqrt(xij**2 + yij**2 + zij**2))/(xij**2 + yij**2 + zij**2)**(3/2) - 4*exp(-xi**2*(xij**2 + yij**2 + zij**2))*xi**3*xij*yij/sqrt(pi) + 2*exp(-xi**2*(xij**2 + yij**2 + zij**2))*xi*xij*yij/(sqrt(pi)*(xij**2 + yij**2 + zij**2)))

In [467]:
def M2r(alpha, beta):
    func = norm(rij)*erf(xi*norm(rij))
    return KroneckerDelta[alpha, beta]*Laplacian(func) - diff( func, rij[alpha], rij[beta])

In [494]:

sympy.limit(sympy.powdenest(M2r(0,0).subs({xij:L, yij:L, zij:L}), force=True), L, 0) ## have to multiply by 1/8*pi*eta

8*xi/sqrt(pi)

In [493]:
sympy.powdenest(M2r(0,0))

-4*xi**3*yij**2*exp(-xi**2*(xij**2 + yij**2 + zij**2))/sqrt(pi) - 4*xi**3*zij**2*exp(-xi**2*(xij**2 + yij**2 + zij**2))/sqrt(pi) + 2*xi*yij**2*exp(-xi**2*(xij**2 + yij**2 + zij**2))/(sqrt(pi)*(xij**2 + yij**2 + zij**2)) + 2*xi*zij**2*exp(-xi**2*(xij**2 + yij**2 + zij**2))/(sqrt(pi)*(xij**2 + yij**2 + zij**2)) + 4*xi*exp(-xi**2*(xij**2 + yij**2 + zij**2))/sqrt(pi) - yij**2*erf(xi*sqrt(xij**2 + yij**2 + zij**2))/(xij**2 + yij**2 + zij**2)**(3/2) - zij**2*erf(xi*sqrt(xij**2 + yij**2 + zij**2))/(xij**2 + yij**2 + zij**2)**(3/2) + 2*erf(xi*sqrt(xij**2 + yij**2 + zij**2))/sqrt(xij**2 + yij**2 + zij**2)